Розробка моделі, здатної ідентифікувати та класифікувати різні рівні токсичності в коментарях, використовуючи можливості BERT (Bidirectional Encoder Representations from Transformers) для аналізу тексту.

Загальний опис проблеми та підходу.

Для навчання наявна дуже незбалансована вибірка і задача з багатоміткової класифікації досить складна. Тому планується застосувати багатозадачний підхід і в моделі виконувати класифікацію у два етапи:

* бінарна класифікація: токсичні - нетоксичні коментарі
* серед токсичних коментарів: багатоміткова класифікація типу токсичності

В рамках цього модулю буде виконано:

* Підготовка датасету до передачі у модель: перетворення форматів для прийняття даних моделлю, виділення міні-вибірки для навчання тестових версій моделі і вибору оптимальної архітектури.
* Розробка і тестування архітектури моделі: на цьому етапі планується виконати декілька варіантів моделі із різною логікою, протестувати чи працює код, виконати навчання підготовлених моделей на міні-вибірці з метою вибору кращої архітектури.
* Робота над покращенням архітектурних рішень у вибраному підході

Навчання моделі на повному наборі тренувальних даних із вибраною архітектурою буде виконано у окремому ноутбуці (другий розділ).



In [1]:
# Перевірка підключення GPU
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


In [2]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Layer, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Precision, Recall


from transformers import TFBertForSequenceClassification, TFBertModel

import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from sklearn.model_selection import train_test_split


**Підготовка тренувальних даних**

In [22]:
data_path = '/kaggle/input/dataset-new/train_data.csv'
df = pd.read_csv(data_path)

# список категорій:
LABEL_COLUMNS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Конвертація токенізованих даних з рядків у масиви
for column in ['input_ids', 'attention_masks']:
    df[column] = df[column].apply(eval).apply(np.array)

# Виділяємо токенізовані вектори та мітки
input_ids = np.stack(df['input_ids'].values)
attention_mask = np.stack(df['attention_masks'].values)
labels = np.array(df[LABEL_COLUMNS].values)
labels = labels.astype('float32')

In [5]:
# Розділення на тренувальну та тестову вибірки (повний набір даних)

train_input_ids, val_input_ids, train_attention_mask, val_attention_mask, train_labels, val_labels = train_test_split(
    input_ids, attention_mask, labels, test_size=0.2, random_state=42
)

In [4]:
# Виділення міні-вибірки 5%

_, mini_input_ids, _, mini_attention_mask, _, mini_labels = train_test_split(
    input_ids, attention_mask, labels, test_size=0.05, random_state=42
)

In [5]:
# розділення міні вибірки на тренувальну та валідаційну
t_input_ids, v_input_ids, t_attention_mask, v_attention_mask, t_labels, v_labels = train_test_split(
    mini_input_ids, mini_attention_mask, mini_labels, test_size=0.2, random_state=42
)


In [8]:
# "Розподіл класів у повній вибірці
print("Розподіл класів у повній вибірці:\n", labels.sum(axis=0))
print("Кількість прикладів нетоксичних коментарів:", (labels.sum(axis=1) == 0).sum())
print("Кількість прикладів у повній вибірці:", len(labels))

# Розподіл класів у міні вибірці
print("Розподіл класів у міні вибірці:\n", mini_labels.sum(axis=0))
print("Кількість прикладів нетоксичних коментарів:", (mini_labels.sum(axis=1) == 0).sum())
print("Кількість прикладів у міні вибірці:",len(mini_labels))

Розподіл класів у повній вибірці:
 [15294.  1595.  8449.   478.  7877.  1405.]
Кількість прикладів нетоксичних коментарів: 143346
Кількість прикладів у повній вибірці: 159571
Розподіл класів у міні вибірці:
 [748.  80. 421.  13. 410.  71.]
Кількість прикладів нетоксичних коментарів: 7181
Кількість прикладів у міні вибірці: 7979


**Підбір архітектури моделі на міні-вибірці**

Для роботи із незбалансованими даними у якості метрики доцільно використовувати F1 метрику.

Проте її використання на пряму (врахування у функції втрат) має певні проблеми і планується використовувати вбудовані функції втрат.

* метрика Ф-1 є нелінійною та залежить від precision та recall. Її оптимізація на пряму через функцію втрат вимагає складних розрахунків які можуть бути не стабільними
* спроби оптимізувати Ф1 на пряму можуть призвести до того, що градієнти стануть надто малими і навчання буде стагнувати
* тому планується додати кастомну метрику Ф-1 для відслідковування, а у якості функції втрат використовувати вбудовані функції (тобто модель при навчанні не буде її оптимізувати на пряму).
* на етапі вибору моделі пріоритет матимуть моделі які показали найкращу динаміку по метриці Ф-1
* у функції втрат планується додати ваги класів для того, щоб врахувати дисбаланс даних



В рамках цього етапу планується реалізувати 3 стратегії побудови моделі:

* Єдина модель із двома незалежними "головами" для бінарної класифікації токсичних-нетоксичних коментарів та для мультиміткової класифікації токсичних коментарів. Обидві голови будуть навчатись на повних вибірках даних. Модель буде мати занижену точність у порівнянні із фактичною за рахунок помилок другої голови на нетоксичних прикладах.
* Єдина модель аналогічна першій, проте друга модель буде навчатись лише на токсичних прикладах для більш сфокусованого навчання.
* Дві послідовні моделі, перша з яких виконає бінарну класифікацію, а друга повністю незалежно навчається лише на токсичних коментарях і виконує їх класифікацію.

Через обмеження обчислювальних ресурсів ми не будемо підбирати параметри моделей та викорастаємо "best prаctices". Також на этапі експериментів ми будемо виконувати лише feature extraction без fine-tuning для економії ресурсів.

За результатами оцінки буде вибрано кращу стратегію та на основі її побудовано фінальну модель та навчено на повній вибірці даних.


In [5]:
# Кастомний шар для інтеграції з BERT
class BertLayer(Layer):
    def __init__(self, pretrained_model_name="bert-base-uncased", trainable=False, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        # Завантажуємо попередньо навчений BERT
        self.bert = TFBertModel.from_pretrained(pretrained_model_name)
        self.bert.trainable = trainable  # Заморожуємо або розморожуємо шари залежно від параметра trainable

    def call(self, inputs):
        # Вхідні дані: input_ids та attention_mask
        input_ids, attention_mask = inputs
        # Передаємо дані через BERT
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state  # Повертаємо тільки last_hidden_state

In [6]:
'''
у якості метрики обрано Ф-1 у зв'язку із незбалансованістю класів. 
Підготуємо функцію для неї
'''

import tensorflow as tf
from tensorflow.keras import backend as K

def f1_metric(y_true, y_pred):
    # Преобразуем в бинарный формат для каждого класса
    y_true = K.cast(y_true, 'int32')
    y_pred = K.cast(K.greater_equal(y_pred, 0.5), 'int32')

    # Вычисляем точность (precision) и полноту (recall)
    true_positive = K.sum(K.cast(y_true * y_pred, 'float32'))
    false_positive = K.sum(K.cast((1 - y_true) * y_pred, 'float32'))
    false_negative = K.sum(K.cast(y_true * (1 - y_pred), 'float32'))

    precision = true_positive / (true_positive + false_positive + K.epsilon())
    recall = true_positive / (true_positive + false_negative + K.epsilon())

    # F1-score = 2 * (precision * recall) / (precision + recall)
    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
    
    return f1

In [26]:
# ваги класів 

# Розподіл класів у повній вибірці
class_distribution = np.array([15294., 1595., 8449., 478., 7877., 1405.])

# Розрахунок ваг класів
total_samples = np.sum(class_distribution)
class_weights = total_samples / (len(class_distribution) * class_distribution)

# Нормалізація ваг
class_weights = class_weights / np.min(class_weights)

# Вивід розрахованих ваг
print(f"Class weights for multilabel_output: {class_weights}")

Class weights for multilabel_output: [ 1.          9.58871473  1.81015505 31.9958159   1.94160213 10.88540925]


In [27]:
# Кастомна функція втрат для врахування ваг класів

def weighted_binary_crossentropy(class_weights):
    def loss(y_true, y_pred):
        # Применяем веса для каждого класса
        weights = tf.reduce_sum(class_weights * y_true, axis=-1)
        # Бинарная кроссэнтропия
        bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        # Взвешиваем loss
        return bce * weights
    return loss

**Перша версія моделі**

In [13]:
# Вхідні дані
input_ids = Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# BERT шар
bert_outputs = BertLayer(trainable=False)([input_ids, attention_mask])

# Пулінг
pooled_output = GlobalAveragePooling1D()(bert_outputs)

# Перша голова - бінарна класифікація
binary_dense = Dense(128, activation="swish")(pooled_output)
binary_dropout = Dropout(0.3)(binary_dense)
binary_output = Dense(1, activation="sigmoid", name="binary_output")(binary_dropout)

# Друга голова - багатоміткова класифікація токсичних коментарів
multilabel_dense = Dense(128, activation="swish")(pooled_output)
multilabel_dropout = Dropout(0.3)(multilabel_dense)
multilabel_output = Dense(6, activation="sigmoid", name="multilabel_output")(multilabel_dropout)

# Модель
model_1 = Model(
    inputs=[input_ids, attention_mask],
    outputs=[binary_output, multilabel_output]
)

class_weights_tensor = tf.constant(class_weights, dtype=tf.float32)

# Компіляція моделі
model_1.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss={
        "binary_output": "binary_crossentropy",
        "multilabel_output": weighted_binary_crossentropy(class_weights_tensor)
    },
    loss_weights={
        "binary_output": 0.5,
        "multilabel_output": 1.0
    },
    metrics={
        "binary_output": ["accuracy"],
        "multilabel_output": ["accuracy", Precision(name="precision"), Recall(name="recall"), f1_metric]
    }
)

model_1.summary()

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_ids (InputLayer)    │ (None, 128)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_mask            │ (None, 128)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_layer (BertLayer)    │ (None, 128, 768)       │              0 │ input_ids[0][0],       │
│                           │                        │                │ attention_mask[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d  │ (None, 768)            │              0 │ bert_layer[0][0]       │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │         98,432 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 128)            │         98,432 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 128)            │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 128)            │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ binary_output (Dense)     │ (None, 1)              │            129 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multilabel_output (Dense) │ (None, 6)              │            774 │ dropout_1[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 197,767 (772.53 KB)

 Trainable params: 197,767 (772.53 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Генерація міток для першої голови (binary_output)
t_binary_labels = np.where(np.all(t_labels == 0, axis=1), 1, 0).astype('float32')
v_binary_labels = np.where(np.all(v_labels == 0, axis=1), 1, 0).astype('float32')

print(f"Shape of t_binary_labels: {t_binary_labels.shape}")
print(f"Example t_binary_labels: {t_binary_labels[:10]}")
print(f"Shape of v_binary_labels: {v_binary_labels.shape}")
print(f"Example v_binary_labels: {v_binary_labels[:10]}")

Shape of t_binary_labels: (6383,)
Example t_binary_labels: [1. 1. 1. 0. 1. 1. 1. 1. 1. 1.]
Shape of v_binary_labels: (1596,)
Example v_binary_labels: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [15]:
# Підготовка даних для двух голов
t_data = {
    "input_ids": t_input_ids,
    "attention_mask": t_attention_mask,
}

v_data = {
    "input_ids": v_input_ids,
    "attention_mask": v_attention_mask,
}

t_labels_combined = {
    "binary_output": t_binary_labels,
    "multilabel_output": t_labels,
}

v_labels_combined = {
    "binary_output": v_binary_labels,
    "multilabel_output": v_labels,
}


# Навчання моделі
history = model_1.fit(
    t_data,
    t_labels_combined,
    validation_data=(v_data, v_labels_combined),
    epochs=5,  
    batch_size=32
)


Epoch 1/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 95s 368ms/step - binary_output_accuracy: 0.8108 - loss: 0.5167 - multilabel_output_accuracy: 0.6573 - multilabel_output_f1_metric: 0.1059 - multilabel_output_precision: 0.0588 - multilabel_output_recall: 0.7679 - val_binary_output_accuracy: 0.8966 - val_loss: 0.4162 - val_multilabel_output_accuracy: 0.9944 - val_multilabel_output_f1_metric: 0.1259 - val_multilabel_output_precision: 0.0692 - val_multilabel_output_recall: 0.9208
Epoch 2/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 57s 284ms/step - binary_output_accuracy: 0.9002 - loss: 0.4566 - multilabel_output_accuracy: 0.9586 - multilabel_output_f1_metric: 0.1179 - multilabel_output_precision: 0.0645 - multilabel_output_recall: 0.9064 - val_binary_output_accuracy: 0.8966 - val_loss: 0.4221 - val_multilabel_output_accuracy: 0.9944 - val_multilabel_output_f1_metric: 0.1264 - val_multilabel_output_precision: 0.0695 - val_multilabel_output_recall: 0.9208
Epoch 3/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 58s 290ms/step - bina

**Друга версія моделі**

***ПРИМІТКА:***

Другий підхід не вдалось реалізувати через вимоги бібліотеки Керас. 

Згідно із логікою запропонованої архітектури модель повина мати два окремі входи: 
* повний набір даних із бінарними мітками для бінарної голови
* частковий набір даних лише із токсичними коментарями та мультимітками для мультиміткової голови і такий набор даних очевидно коротший за перший.

Проте бібіліотека Keras вимагає, щоб всі вхідні дані та вихідні мітки мали однакову довжину. 

Наступний код поданий у закоментованому вигляді для референсу. 


In [16]:
"""
# Вхідні дані для двох голов (окремі входи для кожної голови моделі)
binary_input_ids = Input(shape=(128,), dtype=tf.int32, name="binary_input_ids")
binary_attention_mask = Input(shape=(128,), dtype=tf.int32, name="binary_attention_mask")

multilabel_input_ids = Input(shape=(128,), dtype=tf.int32, name="multilabel_input_ids")
multilabel_attention_mask = Input(shape=(128,), dtype=tf.int32, name="multilabel_attention_mask")

# BERT шар для обох голов 
bert_outputs_binary = BertLayer(trainable=False)([binary_input_ids, binary_attention_mask])
bert_outputs_multilabel = BertLayer(trainable=False)([multilabel_input_ids, multilabel_attention_mask])

# Пулінг для першої голови
pooled_output_binary = GlobalAveragePooling1D()(bert_outputs_binary)
binary_dense = Dense(128, activation="swish")(pooled_output_binary)
binary_dropout = Dropout(0.3)(binary_dense)
binary_output = Dense(1, activation="sigmoid", name="binary_output")(binary_dropout)

# Пулінг для другої голови
pooled_output_multilabel = GlobalAveragePooling1D()(bert_outputs_multilabel)
multilabel_dense = Dense(128, activation="swish")(pooled_output_multilabel)
multilabel_dropout = Dropout(0.3)(multilabel_dense)
multilabel_output = Dense(6, activation="sigmoid", name="multilabel_output")(multilabel_dropout)

# Модель
model_2 = Model(
    inputs=[
        binary_input_ids, 
        binary_attention_mask, 
        multilabel_input_ids, 
        multilabel_attention_mask
    ],
    outputs=[binary_output, multilabel_output]
)

class_weights_tensor = tf.constant(class_weights, dtype=tf.float32)

# Компіляція моделі
model_2.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss={
        "binary_output": "binary_crossentropy",
        "multilabel_output": weighted_binary_crossentropy(class_weights_tensor)
    },
    loss_weights={
        "binary_output": 0.5,
        "multilabel_output": 1.0
    },
    metrics={
        "binary_output": ["accuracy"],
        "multilabel_output": ["accuracy", Precision(name="precision"), Recall(name="recall"), f1_metric]
    }
)

model_2.summary()
"""

'\n# Вхідні дані для двох голов (окремі входи для кожної голови моделі)\nbinary_input_ids = Input(shape=(128,), dtype=tf.int32, name="binary_input_ids")\nbinary_attention_mask = Input(shape=(128,), dtype=tf.int32, name="binary_attention_mask")\n\nmultilabel_input_ids = Input(shape=(128,), dtype=tf.int32, name="multilabel_input_ids")\nmultilabel_attention_mask = Input(shape=(128,), dtype=tf.int32, name="multilabel_attention_mask")\n\n# BERT шар для обох голов \nbert_outputs_binary = BertLayer(trainable=False)([binary_input_ids, binary_attention_mask])\nbert_outputs_multilabel = BertLayer(trainable=False)([multilabel_input_ids, multilabel_attention_mask])\n\n# Пулінг для першої голови\npooled_output_binary = GlobalAveragePooling1D()(bert_outputs_binary)\nbinary_dense = Dense(128, activation="swish")(pooled_output_binary)\nbinary_dropout = Dropout(0.3)(binary_dense)\nbinary_output = Dense(1, activation="sigmoid", name="binary_output")(binary_dropout)\n\n# Пулінг для другої голови\npooled_

In [17]:
"""
# Генерація міток для першої голови (binary_output) - без змін у порівнянні із першою версією
t_binary_labels = np.where(np.all(t_labels == 0, axis=1), 1, 0).astype('float32')
v_binary_labels = np.where(np.all(v_labels == 0, axis=1), 1, 0).astype('float32')
"""

"\n# Генерація міток для першої голови (binary_output) - без змін у порівнянні із першою версією\nt_binary_labels = np.where(np.all(t_labels == 0, axis=1), 1, 0).astype('float32')\nv_binary_labels = np.where(np.all(v_labels == 0, axis=1), 1, 0).astype('float32')\n"

In [18]:
"""
# Підготовка даних для двух голов (змінено для другої версії моделі)

# Видбірка токсичних коментарів для другої голови
t_toxic_indices = np.any(t_labels == 1, axis=1)
v_toxic_indices = np.any(v_labels == 1, axis=1)

# Вхідні дані лише для токсичних прикладів
t_toxic_input_ids = t_input_ids[t_toxic_indices]
t_toxic_attention_mask = t_attention_mask[t_toxic_indices]
t_toxic_labels = t_labels[t_toxic_indices]

v_toxic_input_ids = v_input_ids[v_toxic_indices]
v_toxic_attention_mask = v_attention_mask[v_toxic_indices]
v_toxic_labels = v_labels[v_toxic_indices]

# Підготовка даних для двох голов
# Перша голова отримує повну вибірку
t_data_binary = {
    "input_ids": t_input_ids,
    "attention_mask": t_attention_mask,
}

v_data_binary = {
    "input_ids": v_input_ids,
    "attention_mask": v_attention_mask,
}

# Друга голова отримує лише токсичні приклади
t_data_multilabel = {
    "input_ids": t_toxic_input_ids,
    "attention_mask": t_toxic_attention_mask,
}

v_data_multilabel = {
    "input_ids": v_toxic_input_ids,
    "attention_mask": v_toxic_attention_mask,
}

# Підготовка міток для двох голов
t_labels_combined = {
    "binary_output": t_binary_labels,
    "multilabel_output": t_toxic_labels,
}

v_labels_combined = {
    "binary_output": v_binary_labels,
    "multilabel_output": v_toxic_labels,
}
"""

'\n# Підготовка даних для двух голов (змінено для другої версії моделі)\n\n# Видбірка токсичних коментарів для другої голови\nt_toxic_indices = np.any(t_labels == 1, axis=1)\nv_toxic_indices = np.any(v_labels == 1, axis=1)\n\n# Вхідні дані лише для токсичних прикладів\nt_toxic_input_ids = t_input_ids[t_toxic_indices]\nt_toxic_attention_mask = t_attention_mask[t_toxic_indices]\nt_toxic_labels = t_labels[t_toxic_indices]\n\nv_toxic_input_ids = v_input_ids[v_toxic_indices]\nv_toxic_attention_mask = v_attention_mask[v_toxic_indices]\nv_toxic_labels = v_labels[v_toxic_indices]\n\n# Підготовка даних для двох голов\n# Перша голова отримує повну вибірку\nt_data_binary = {\n    "input_ids": t_input_ids,\n    "attention_mask": t_attention_mask,\n}\n\nv_data_binary = {\n    "input_ids": v_input_ids,\n    "attention_mask": v_attention_mask,\n}\n\n# Друга голова отримує лише токсичні приклади\nt_data_multilabel = {\n    "input_ids": t_toxic_input_ids,\n    "attention_mask": t_toxic_attention_mask,\

In [19]:
"""
history = model_2.fit(
    {
        "binary_input_ids": t_input_ids,  # Повна вибірка для першої голови
        "binary_attention_mask": t_attention_mask,
        "multilabel_input_ids": t_toxic_input_ids,  # Токсичні дані для другої голови
        "multilabel_attention_mask": t_toxic_attention_mask,
    },
    {
        "binary_output": t_binary_labels,  # Повні мітки для першої голови
        "multilabel_output": t_toxic_labels,  # Мітки лише для токсичних прикладів
    },
    validation_data=(
        {
            "binary_input_ids": v_input_ids,  # Повна вибірка для першої голови
            "binary_attention_mask": v_attention_mask,
            "multilabel_input_ids": v_toxic_input_ids,  # Токсичні дані для другої голови
            "multilabel_attention_mask": v_toxic_attention_mask,
        },
        {
            "binary_output": v_binary_labels,  # Повні мітки для першої голови
            "multilabel_output": v_toxic_labels,  # Мітки лише для токсичних прикладів
        }
    ),
    epochs=5,
    batch_size=32
)
"""

'\nhistory = model_2.fit(\n    {\n        "binary_input_ids": t_input_ids,  # Повна вибірка для першої голови\n        "binary_attention_mask": t_attention_mask,\n        "multilabel_input_ids": t_toxic_input_ids,  # Токсичні дані для другої голови\n        "multilabel_attention_mask": t_toxic_attention_mask,\n    },\n    {\n        "binary_output": t_binary_labels,  # Повні мітки для першої голови\n        "multilabel_output": t_toxic_labels,  # Мітки лише для токсичних прикладів\n    },\n    validation_data=(\n        {\n            "binary_input_ids": v_input_ids,  # Повна вибірка для першої голови\n            "binary_attention_mask": v_attention_mask,\n            "multilabel_input_ids": v_toxic_input_ids,  # Токсичні дані для другої голови\n            "multilabel_attention_mask": v_toxic_attention_mask,\n        },\n        {\n            "binary_output": v_binary_labels,  # Повні мітки для першої голови\n            "multilabel_output": v_toxic_labels,  # Мітки лише для токсичн

**Третя версія моделі (дві послідовні моделі)**

In [20]:
# Модель для бінарної класифікації, навчена на повних даних 

# Вхідні дані
input_ids = Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# BERT шар
bert_outputs = BertLayer(trainable=False)([input_ids, attention_mask])

# Пулінг
pooled_output = GlobalAveragePooling1D()(bert_outputs)

# бінарна класифікація
binary_dense = Dense(128, activation="swish")(pooled_output)
binary_dropout = Dropout(0.3)(binary_dense)
binary_output = Dense(1, activation="sigmoid", name="binary_output")(binary_dropout)

# Модель
model_3_1 = Model(
    inputs=[input_ids, attention_mask],
    outputs=[binary_output]
)

# Компіляція моделі
model_3_1.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model_3_1.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_ids (InputLayer)    │ (None, 128)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_mask            │ (None, 128)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_layer_1 (BertLayer)  │ (None, 128, 768)       │              0 │ input_ids[0][0],       │
│                           │                        │                │ attention_mask[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 768)            │              0 │ bert_layer_1[0][0]     │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 128)            │         98,432 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 128)            │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ binary_output (Dense)     │ (None, 1)              │            129 │ dropout_2[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 98,561 (385.00 KB)

 Trainable params: 98,561 (385.00 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Генерація міток для бінарної моделі
t_binary_labels = np.where(np.all(t_labels == 0, axis=1), 1, 0).astype('float32')
v_binary_labels = np.where(np.all(v_labels == 0, axis=1), 1, 0).astype('float32')

In [22]:
# Навчання моделі
history_3_1 = model_3_1.fit(
    t_data,
    t_binary_labels,
    validation_data=(v_data, v_binary_labels),
    epochs=5,  
    batch_size=32
)

Epoch 1/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 93s 380ms/step - accuracy: 0.8959 - loss: 0.3490 - val_accuracy: 0.8966 - val_loss: 0.3319
Epoch 2/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 59s 294ms/step - accuracy: 0.9008 - loss: 0.3294 - val_accuracy: 0.8966 - val_loss: 0.3352
Epoch 3/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 57s 286ms/step - accuracy: 0.9008 - loss: 0.3214 - val_accuracy: 0.8966 - val_loss: 0.3313
Epoch 4/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 57s 287ms/step - accuracy: 0.9029 - loss: 0.3178 - val_accuracy: 0.8966 - val_loss: 0.3355
Epoch 5/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 57s 288ms/step - accuracy: 0.9050 - loss: 0.3142 - val_accuracy: 0.8966 - val_loss: 0.3318


In [23]:
# Модель для мультиміткової класифікації, навчена на токсичних коментарях 

# Вхідні дані
input_ids = Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# BERT шар
bert_outputs = BertLayer(trainable=False)([input_ids, attention_mask])

# Пулінг
pooled_output = GlobalAveragePooling1D()(bert_outputs)

# багатоміткова класифікація токсичних коментарів
multilabel_dense = Dense(128, activation="swish")(pooled_output)
multilabel_dropout = Dropout(0.3)(multilabel_dense)
multilabel_output = Dense(6, activation="sigmoid", name="multilabel_output")(multilabel_dropout)

# Модель
model_3_2 = Model(
    inputs=[input_ids, attention_mask],
    outputs=[multilabel_output]
)

class_weights_tensor = tf.constant(class_weights, dtype=tf.float32)

# Компіляція моделі
model_3_2.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss=weighted_binary_crossentropy(class_weights_tensor),
    metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), f1_metric]
)

model_3_2.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_ids (InputLayer)    │ (None, 128)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_mask            │ (None, 128)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_layer_2 (BertLayer)  │ (None, 128, 768)       │              0 │ input_ids[0][0],       │
│                           │                        │                │ attention_mask[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 768)            │              0 │ bert_layer_2[0][0]     │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 128)            │         98,432 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 128)            │              0 │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multilabel_output (Dense) │ (None, 6)              │            774 │ dropout_3[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 99,206 (387.52 KB)

 Trainable params: 99,206 (387.52 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
# Підготовка даних для мультиміткової моделі (лише токсичні коментарі)

# Видбірка токсичних коментарів
t_toxic_indices = np.any(t_labels == 1, axis=1)
v_toxic_indices = np.any(v_labels == 1, axis=1)

# Вхідні дані лише для токсичних прикладів
t_toxic_input_ids = t_input_ids[t_toxic_indices]
t_toxic_attention_mask = t_attention_mask[t_toxic_indices]
t_toxic_labels = t_labels[t_toxic_indices]

v_toxic_input_ids = v_input_ids[v_toxic_indices]
v_toxic_attention_mask = v_attention_mask[v_toxic_indices]
v_toxic_labels = v_labels[v_toxic_indices]

In [25]:
# Навчання моделі
history_3_2 = model_3_2.fit(
    {
        'input_ids': t_toxic_input_ids,
        'attention_mask': t_toxic_attention_mask
    },
    t_toxic_labels,
    validation_data=(
        {
            'input_ids': v_toxic_input_ids,
            'attention_mask': v_toxic_attention_mask
        },
        v_toxic_labels),
    epochs=5,  
    batch_size=32
)

Epoch 1/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.1027 - f1_metric: 0.4004 - loss: 4.6195 - precision: 0.3323 - recall: 0.5072 - val_accuracy: 0.9030 - val_f1_metric: 0.7214 - val_loss: 2.7634 - val_precision: 0.6188 - val_recall: 0.9180
Epoch 2/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 292ms/step - accuracy: 0.5983 - f1_metric: 0.7020 - loss: 2.7162 - precision: 0.5813 - recall: 0.8891 - val_accuracy: 0.9455 - val_f1_metric: 0.7681 - val_loss: 2.4247 - val_precision: 0.6788 - val_recall: 0.9180
Epoch 3/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 302ms/step - accuracy: 0.8162 - f1_metric: 0.7461 - loss: 2.5293 - precision: 0.6350 - recall: 0.9063 - val_accuracy: 0.9455 - val_f1_metric: 0.7681 - val_loss: 2.4038 - val_precision: 0.6788 - val_recall: 0.9180
Epoch 4/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 306ms/step - accuracy: 0.9092 - f1_metric: 0.7360 - loss: 2.7065 - precision: 0.6222 - recall: 0.9053 - val_accuracy: 0.9455 - val_f1_metric: 0.7681 - val_loss: 2.4030 - val_precision: 0.6788 - val_recal

**Порівняння точності двох моделей та виконання прогнозів**

Для прогнозування і оцінки тестових моделей використаємо валідаційну міні вибірку:

v_input_ids, v_attention_mask, v_labels

In [26]:
# Оцінка першої версії моделі

# Додання класу нетоксичних коментарів (всі нулі)
all_zeros_class = np.all(v_labels == 0, axis=1).astype(int)  
y_test_expanded = np.hstack((v_labels, all_zeros_class.reshape(-1, 1)))  

# Отримання прогнозів
predictions_test = model_1.predict(
    {'input_ids': v_input_ids, 'attention_mask': v_attention_mask},
    batch_size=64
)

# Розділення прогнозів по головам
binary_output = predictions_test[0]  
multilabel_output = predictions_test[1]  

# Перетворення бінарного виходу
binary_predictions = (binary_output > 0.5).astype(int)  # Перетворення в 0 або 1

# Перетворення мультиміткового виходу
multilabel_predictions = (multilabel_output > 0.5).astype(int)  # 

# Формування вектору результатів
final_predictions = []
for binary, multilabel in zip(binary_predictions, multilabel_predictions):
    if binary == 1:
        # Якщо binary_output = 1, то всі інші мітки = 0
        final_predictions.append([0, 0, 0, 0, 0, 0, 1])  # Індекс 6 для binary_output
    else:
        # Якщо binary_output = 0, то використовуємо multilabel_output
        multilabel_result = multilabel.tolist() + [0]  # Додаємо 0 замість binary_output
        final_predictions.append(multilabel_result)

final_predictions = np.array(final_predictions)

# Оцінка моделі
print("\nКласифікаційний звіт для першого підходу:\n")
print(classification_report(y_test_expanded, final_predictions, target_names=[
    "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate", "non_toxic"
]))

# Побудова багатоміткової матриці помилок
conf_matrices = multilabel_confusion_matrix(y_test_expanded, final_predictions)

# Приклад виводу (наприклад для "toxic")
print("Confusion matrix for 'toxic':")
print(conf_matrices[0])

25/25 ━━━━━━━━━━━━━━━━━━━━ 25s 771ms/step

Класифікаційний звіт для першого підходу:

               precision    recall  f1-score   support

        toxic       0.00      0.00      0.00       156
 severe_toxic       0.00      0.00      0.00        18
      obscene       0.00      0.00      0.00        89
       threat       0.00      0.00      0.00         3
       insult       0.00      0.00      0.00        91
identity_hate       0.00      0.00      0.00         9
    non_toxic       0.90      1.00      0.95      1431

    micro avg       0.90      0.80      0.84      1797
    macro avg       0.13      0.14      0.14      1797
 weighted avg       0.71      0.80      0.75      1797
  samples avg       0.90      0.90      0.90      1797

Confusion matrix for 'toxic':
[[1440    0]
 [ 156    0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
# Оцінка другої версії моделі

# Додання класу нетоксичних коментарів (всі нулі)
all_zeros_class = np.all(v_labels == 0, axis=1).astype(int)  
y_test_expanded = np.hstack((v_labels, all_zeros_class.reshape(-1, 1)))  

# Отримання прогнозів для першої моделі
binary_predictions = model_3_1.predict(
    {'input_ids': v_input_ids, 'attention_mask': v_attention_mask},
    batch_size=64
)
binary_predictions = (binary_output > 0.5).astype(int)  # Перетворення в 0 або 1

final_predictions = []

# Прододимо по кодному прикладу валідаційних даних
for i in range(len(v_input_ids)):
    binary_prediction = binary_predictions[i]  # Прогноз бінарної моделі для поточного приклада

    if binary_prediction == 1:
        # Якщо коментар не токсичний, формуємо фінальний вектор
        final_predictions.append([0, 0, 0, 0, 0, 0, 1])  # Всі нулі + 1 на останьому індексі
    else:
        # Якщо коментар токсичний, формуємо прогноз мультимітковою моделлю
        toxic_input_ids = v_input_ids[i].reshape(1, -1)  # Приклад в форматі (1, 128)
        toxic_attention_mask = v_attention_mask[i].reshape(1, -1)

        # Прогноз мультимітковою моделлю
        multilabel_prediction = model_3_2.predict(
            {'input_ids': toxic_input_ids, 'attention_mask': toxic_attention_mask},
            batch_size=1
        )

        # Перетворення прогнозів
        multilabel_result = (multilabel_prediction > 0.5).astype(int).flatten().tolist()
        multilabel_result.append(0)  # Дадаємо 0 в останній індекс 

        # Додаємо результат у фінальні прогнози
        final_predictions.append(multilabel_result)

# Перетворення фінальних прогнозів в numpy-масив
final_predictions = np.array(final_predictions)

# Оцінка моделі
print("\nКласифікаційний звіт для другого підходу моделі:\n")
print(classification_report(y_test_expanded, final_predictions, target_names=[
    "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate", "non_toxic"
]))

# Побудова багатоміткової матриці помилок
conf_matrices = multilabel_confusion_matrix(y_test_expanded, final_predictions)

# Приклад виводу (наприклад для "toxic")
print("Confusion matrix for 'toxic':")
print(conf_matrices[0])

25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 729ms/step

Класифікаційний звіт для другого підходу моделі:

               precision    recall  f1-score   support

        toxic       0.00      0.00      0.00       156
 severe_toxic       0.00      0.00      0.00        18
      obscene       0.00      0.00      0.00        89
       threat       0.00      0.00      0.00         3
       insult       0.00      0.00      0.00        91
identity_hate       0.00      0.00      0.00         9
    non_toxic       0.90      1.00      0.95      1431

    micro avg       0.90      0.80      0.84      1797
    macro avg       0.13      0.14      0.14      1797
 weighted avg       0.71      0.80      0.75      1797
  samples avg       0.90      0.90      0.90      1797

Confusion matrix for 'toxic':
[[1440    0]
 [ 156    0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Висновки**

Обидві моделі показали незадовільні результати. Оскільки логіка побудови моделей вірна, проблема саме в тренувальних даних та їх незбалансованості. 

Далі ми спробуємо виконати обробку тренувальної вибірки (андерсемплінг та оверсемплінг), та покращити архітектуру моделей. 

Оскільки третій підхід (дві послідовні моделі) виявляється більш легким для відстежування результатів кожного етапу, сконцентруємось на ньому.


**Розробка покращеної бінарної моделі**

Поточна версія бінарної моделі протемонструвала схильність до визначення всіх коментарів як нетоксичних. Оскільки тренувальна вибірка незбалансована і має 90% нетоксичних прикладів, модель продемонстувала оманливо високу точність, проте вона не здатна виявляти токсичні коментарі. 

Для покращення результату спробуємо виконати балансування тренувальних даних. 

In [9]:
from imblearn.over_sampling import SMOTE

t_features = np.hstack((t_input_ids, t_attention_mask))

smote = SMOTE(random_state=42)
t_data_resampled, t_binary_labels_resampled = smote.fit_resample(t_features, t_binary_labels)

In [10]:
# До SMOTEENN
print("До обробки:")
print(f"Нетоксичні: {np.sum(t_binary_labels == 0)}")
print(f"Токсичні: {np.sum(t_binary_labels == 1)}")

# Після SMOTEENN
print("\nПісля обробки:")
print(f"Нетоксичні: {np.sum(t_binary_labels_resampled == 0)}")
print(f"Токсичні: {np.sum(t_binary_labels_resampled == 1)}")

До обробки:
Нетоксичні: 633
Токсичні: 5750

Після обробки:
Нетоксичні: 5750
Токсичні: 5750


In [11]:
# Зворотне перетворення на t_input_ids і t_attention_mask

# Вихідні розміри t_input_ids и t_attention_mask
input_ids_size = t_input_ids.shape[1]
attention_mask_size = t_attention_mask.shape[1]

# Зворотній розподіл
t_input_ids_resampled = t_data_resampled[:, :input_ids_size]
t_attention_mask_resampled = t_data_resampled[:, input_ids_size:]

In [12]:
t_data = {
    "input_ids": t_input_ids_resampled,
    "attention_mask": t_attention_mask_resampled,
}

v_data = {
    "input_ids": v_input_ids,
    "attention_mask": v_attention_mask,
}

In [13]:
# Модель для бінарної класифікації

# Вхідні дані
input_ids = Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# BERT шар
bert_outputs = BertLayer(trainable=False)([input_ids, attention_mask])

# Пулінг
pooled_output = GlobalAveragePooling1D()(bert_outputs)

# бінарна класифікація
binary_dense = Dense(128, activation="swish")(pooled_output)
binary_dropout = Dropout(0.3)(binary_dense)
binary_output = Dense(1, activation="sigmoid", name="binary_output")(binary_dropout)

# Модель
model_3_1 = Model(
    inputs=[input_ids, attention_mask],
    outputs=[binary_output]
)

# Компіляція моделі
model_3_1.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model_3_1.summary()

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_ids (InputLayer)    │ (None, 128)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_mask            │ (None, 128)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_layer (BertLayer)    │ (None, 128, 768)       │              0 │ input_ids[0][0],       │
│                           │                        │                │ attention_mask[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d  │ (None, 768)            │              0 │ bert_layer[0][0]       │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │         98,432 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 128)            │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ binary_output (Dense)     │ (None, 1)              │            129 │ dropout[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 98,561 (385.00 KB)

 Trainable params: 98,561 (385.00 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Навчання моделі
history_3_1 = model_3_1.fit(
    t_data,
    t_binary_labels_resampled,
    validation_data=(v_data, v_binary_labels),
    epochs=5,  
    batch_size=32
)

Epoch 1/5
360/360 ━━━━━━━━━━━━━━━━━━━━ 132s 314ms/step - accuracy: 0.5502 - loss: 0.6985 - val_accuracy: 0.5013 - val_loss: 0.7121
Epoch 2/5
360/360 ━━━━━━━━━━━━━━━━━━━━ 108s 301ms/step - accuracy: 0.5769 - loss: 0.6780 - val_accuracy: 0.5069 - val_loss: 0.7238
Epoch 3/5
360/360 ━━━━━━━━━━━━━━━━━━━━ 107s 298ms/step - accuracy: 0.5858 - loss: 0.6710 - val_accuracy: 0.5088 - val_loss: 0.6964
Epoch 4/5
360/360 ━━━━━━━━━━━━━━━━━━━━ 108s 301ms/step - accuracy: 0.5937 - loss: 0.6701 - val_accuracy: 0.5363 - val_loss: 0.6824
Epoch 5/5
360/360 ━━━━━━━━━━━━━━━━━━━━ 108s 301ms/step - accuracy: 0.5901 - loss: 0.6704 - val_accuracy: 0.5746 - val_loss: 0.6595


In [15]:
# Отримання прогнозів для першої моделі
binary_predictions = model_3_1.predict(
    {'input_ids': v_input_ids, 'attention_mask': v_attention_mask},
    batch_size=64
)

# Преобразуем результат в NumPy-массив, если это необходимо
binary_predictions = (binary_predictions > 0.5).astype(int)  # Перетворення в 0 або 1

# Розподіл прогнозів бінарної моделі
unique, counts = np.unique(binary_predictions, return_counts=True)
binary_distribution = dict(zip(unique, counts))

print("Розподіл міток:")
print(f"Нетоксичні (1): {binary_distribution.get(1, 0)}")
print(f"Токсичні (0): {binary_distribution.get(0, 0)}")

25/25 ━━━━━━━━━━━━━━━━━━━━ 26s 821ms/step
Розподіл міток:
Нетоксичні (1): 934
Токсичні (0): 662


In [40]:
# Розподіл істинних міток
unique, counts = np.unique(v_binary_labels, return_counts=True)
true_distribution = dict(zip(unique, counts))

print("Розподіл істинних міток:")
print(f"Нетоксичні (1): {true_distribution.get(1, 0)}")
print(f"Токсичні (0): {true_distribution.get(0, 0)}")


Розподіл істинних міток:
Нетоксичні (1): 1431
Токсичні (0): 165


In [16]:
# Оцінка моделі
print("\nКласифікаційний звіт для бінарної моделі:\n")
print(classification_report(v_binary_labels, binary_predictions, target_names=[
    "non_toxic", "toxic"
]))


Класифікаційний звіт для бінарної моделі:

              precision    recall  f1-score   support

   non_toxic       0.11      0.45      0.18       165
       toxic       0.90      0.59      0.71      1431

    accuracy                           0.57      1596
   macro avg       0.51      0.52      0.45      1596
weighted avg       0.82      0.57      0.66      1596



Досягненням є те, що тепер модель здатна визначати як токсичні, так у нетоксичні коментарі. Проте на цьому етапі модель демонструє схильність до хибного віднесення коментарів до токсичних. 

Треба враховувати що поточний цикл навчання виконується із вибіркою всього у 5% від загальної. При навчані на повних даних, модель матиме більше прикладів та теоретично краще зможе розрізняти класи коментарів.

Виконанаємо фінтюнінг, щоб побачити чи є динаміка на покращення результатів. 

У тестовій моделі розморозимо лише 2 верхніх шари БЕРТ. При навчанні на повній вибірці будемо розморожувати 4. 

In [17]:
# Разморозка останніх 2-х шарів BERT
for layer in model_3_1.layers:
    if isinstance(layer, BertLayer):  
        for bert_layer in layer.bert.bert.encoder.layer[-2:]:  
            bert_layer.trainable = True

model_3_1.compile(
    optimizer=Adam(learning_rate=1e-5),  # Низкий learning rate для фінтюнинга
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [18]:
# Навчання моделі
history_3_1 = model_3_1.fit(
    t_data,
    t_binary_labels_resampled,
    validation_data=(v_data, v_binary_labels),
    epochs=3,  
    batch_size=32
)

Epoch 1/3
360/360 ━━━━━━━━━━━━━━━━━━━━ 142s 348ms/step - accuracy: 0.6053 - loss: 0.6616 - val_accuracy: 0.5783 - val_loss: 0.6544
Epoch 2/3
360/360 ━━━━━━━━━━━━━━━━━━━━ 109s 303ms/step - accuracy: 0.5980 - loss: 0.6649 - val_accuracy: 0.5551 - val_loss: 0.6753
Epoch 3/3
360/360 ━━━━━━━━━━━━━━━━━━━━ 108s 300ms/step - accuracy: 0.5996 - loss: 0.6631 - val_accuracy: 0.5909 - val_loss: 0.6521


In [19]:
# Отримання прогнозів для першої моделі
binary_predictions = model_3_1.predict(
    {'input_ids': v_input_ids, 'attention_mask': v_attention_mask},
    batch_size=64
)

# Перетворимо результат в NumPy-массив, якщо це необхідно
binary_predictions = (binary_predictions > 0.5).astype(int)  # Перетворення в 0 або 1

# Розподіл прогнозів бінарної моделі
unique, counts = np.unique(binary_predictions, return_counts=True)
binary_distribution = dict(zip(unique, counts))

print("Розподіл міток:")
print(f"Нетоксичні (1): {binary_distribution.get(1, 0)}")
print(f"Токсичні (0): {binary_distribution.get(0, 0)}")

25/25 ━━━━━━━━━━━━━━━━━━━━ 24s 745ms/step
Розподіл міток:
Нетоксичні (1): 970
Токсичні (0): 626


In [20]:
# Оцінка моделі
print("\nКласифікаційний звіт для бінарної моделі:\n")
print(classification_report(v_binary_labels, binary_predictions, target_names=[
    "non_toxic", "toxic"
]))


Класифікаційний звіт для бінарної моделі:

              precision    recall  f1-score   support

   non_toxic       0.11      0.42      0.17       165
       toxic       0.90      0.61      0.73      1431

    accuracy                           0.59      1596
   macro avg       0.51      0.51      0.45      1596
weighted avg       0.82      0.59      0.67      1596



Фінтюнінг не показав суттєвого покращення. Проте можна припустити, що проблема саме в недостатньості тренувальних даних. Модель БЕРТ краще працює із великими масивами даних, тому при передачі на навчання 159 тис. прикладів очікується покращення результатів. 

Приймаємо поточну архітектури бінарної моделі у відповідності до останніх змін. 

**Багатоміткова модель**

Багатоміткова модель продемонструвала схильність присвоювати мітки класів для найбільш поширених класів та інгорувати рідкі класи. 

Потенційні проблеми:
* надмала тренувальна вибірка. Ми передаємо моделі лише токсичні приклади, а це всього близько 800 прикладів. Цього недостатньо щоб модель змогла виділити характеристики класів.
* кастомна функція втрат із застосуванням ваг може не працювати як очікуюється і необхідно виконати балансування даних замість використання ваг класів.
* більш глибокі проблеми із архітектурою моделі (необхідна більш складна архітектура).

Тому плануємо спробувати:
* передати у поточну архітектуру моделі повну вибірку токсичних коментарів
* якщо перший підхід не дасть результатів, спробуємо збалансувати вибірку та розробити архітектуру моделі без кастомної функції

In [23]:
# Розділення на тренувальну та тестову вибірки (повний набір даних)

t_input_ids, v_input_ids, t_attention_mask, v_attention_mask, t_labels, v_labels = train_test_split(
    input_ids, attention_mask, labels, test_size=0.2, random_state=42
)


In [24]:
# Підготовка даних для мультиміткової моделі (лише токсичні коментарі)

# Видбірка токсичних коментарів
t_toxic_indices = np.any(t_labels == 1, axis=1)
v_toxic_indices = np.any(v_labels == 1, axis=1)

# Вхідні дані лише для токсичних прикладів
t_toxic_input_ids = t_input_ids[t_toxic_indices]
t_toxic_attention_mask = t_attention_mask[t_toxic_indices]
t_toxic_labels = t_labels[t_toxic_indices]

v_toxic_input_ids = v_input_ids[v_toxic_indices]
v_toxic_attention_mask = v_attention_mask[v_toxic_indices]
v_toxic_labels = v_labels[v_toxic_indices]

In [25]:
print(len(t_toxic_labels))
print(len(v_toxic_labels))


12981
3244


In [28]:
# Модель для мультиміткової класифікації, навчена на токсичних коментарях 

# Вхідні дані
input_ids = Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# BERT шар
bert_outputs = BertLayer(trainable=False)([input_ids, attention_mask])

# Пулінг
pooled_output = GlobalAveragePooling1D()(bert_outputs)

# багатоміткова класифікація токсичних коментарів
multilabel_dense = Dense(128, activation="swish")(pooled_output)
multilabel_dropout = Dropout(0.3)(multilabel_dense)
multilabel_output = Dense(6, activation="sigmoid", name="multilabel_output")(multilabel_dropout)

# Модель
model_3_2 = Model(
    inputs=[input_ids, attention_mask],
    outputs=[multilabel_output]
)

class_weights_tensor = tf.constant(class_weights, dtype=tf.float32)

# Компіляція моделі
model_3_2.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss=weighted_binary_crossentropy(class_weights_tensor),
    metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), f1_metric]
)

model_3_2.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_ids (InputLayer)    │ (None, 128)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_mask            │ (None, 128)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_layer_1 (BertLayer)  │ (None, 128, 768)       │              0 │ input_ids[0][0],       │
│                           │                        │                │ attention_mask[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 768)            │              0 │ bert_layer_1[0][0]     │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 128)            │         98,432 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 128)            │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multilabel_output (Dense) │ (None, 6)              │            774 │ dropout_1[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 99,206 (387.52 KB)

 Trainable params: 99,206 (387.52 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
# Навчання моделі
history_3_2 = model_3_2.fit(
    {
        'input_ids': t_toxic_input_ids,
        'attention_mask': t_toxic_attention_mask
    },
    t_toxic_labels,
    validation_data=(
        {
            'input_ids': v_toxic_input_ids,
            'attention_mask': v_toxic_attention_mask
        },
        v_toxic_labels),
    epochs=3, # лише три епохи для економії ресурсів у тестовому запуску моделі
    batch_size=32
)

Epoch 1/3
406/406 ━━━━━━━━━━━━━━━━━━━━ 151s 326ms/step - accuracy: 0.7765 - f1_metric: 0.7017 - loss: 3.0252 - precision: 0.5997 - recall: 0.8546 - val_accuracy: 0.9420 - val_f1_metric: 0.7589 - val_loss: 2.7242 - val_precision: 0.6561 - val_recall: 0.9026
Epoch 2/3
406/406 ━━━━━━━━━━━━━━━━━━━━ 122s 301ms/step - accuracy: 0.9400 - f1_metric: 0.7480 - loss: 2.9089 - precision: 0.6399 - recall: 0.9033 - val_accuracy: 0.9420 - val_f1_metric: 0.7589 - val_loss: 2.7107 - val_precision: 0.6561 - val_recall: 0.9026
Epoch 3/3
406/406 ━━━━━━━━━━━━━━━━━━━━ 123s 303ms/step - accuracy: 0.9400 - f1_metric: 0.7478 - loss: 2.9619 - precision: 0.6407 - recall: 0.9006 - val_accuracy: 0.9420 - val_f1_metric: 0.7589 - val_loss: 2.6916 - val_precision: 0.6561 - val_recall: 0.9026


In [30]:
# Прогнози мультиміткової моделі на валідаціних даних
multilabel_predictions = model_3_2.predict(
    {'input_ids': v_toxic_input_ids, 'attention_mask': v_toxic_attention_mask},
    batch_size=64
)

# Перетворюємо прогнози на бінарні мітки
multilabel_predictions = (multilabel_predictions > 0.5).astype(int)
# Сумуємо значення для кожної мітки
toxic_label_counts = multilabel_predictions.sum(axis=0)

# Мітки токсичності
labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

print("\nРозподіл міток багатоміткової моделі:")
for i, label in enumerate(labels):
    print(f"{label}: {toxic_label_counts[i]}")


51/51 ━━━━━━━━━━━━━━━━━━━━ 35s 603ms/step

Розподіл міток багатоміткової моделі:
toxic: 3244
severe_toxic: 0
obscene: 3244
threat: 0
insult: 3244
identity_hate: 0


In [31]:
# Істиний розподіл міток
true_label_counts = v_toxic_labels.sum(axis=0)

print("\nРозподіл істиних міток:")
for i, label in enumerate(labels):
    print(f"{label}: {true_label_counts[i]}")


Розподіл істиних міток:
toxic: 3056.0
severe_toxic: 321.0
obscene: 1715.0
threat: 74.0
insult: 1614.0
identity_hate: 294.0


Модель все ще не розрізняє рідкі класи та надлишково прогнозує часті класи. Тобто проблема не з обсягом даних, а з архітектурою. 

Спробуємо виконати балансування даних та відмовитись від кастомної функції втрат. 

In [35]:
# Розподіл класів у тренувальній вибірці
class_distribution = t_toxic_labels.sum(axis=0)
print("Розподіл класів у тренувальній вибірці:", class_distribution)

Розподіл класів у тренувальній вибірці: [12238.  1274.  6734.   404.  6263.  1111.]


In [36]:
# Балансування даних 

from sklearn.utils import resample

def balance_data(input_ids, attention_mask, labels):
    # Створюємо датафрейм для зручної обробки
    data = pd.DataFrame({
        'input_ids': list(input_ids),
        'attention_mask': list(attention_mask),
        'labels': list(labels)
    })
    
    # Розділяємо дані по класам
    classes = [data[data['labels'].apply(lambda x: x[i] == 1)] for i in range(labels.shape[1])]

    # Находимо середню кількість даних в класі
    avg_class_size = int(np.mean([len(cls) for cls in classes]))

    # Oversampling для кожного класу
    balanced_classes = []
    for cls in classes:
        balanced_classes.append(resample(cls, replace=True, n_samples=avg_class_size, random_state=42))
    
    # Об'єднуємо збалансовані дані
    balanced_data = pd.concat(balanced_classes)
    return (
        np.stack(balanced_data['input_ids'].values),
        np.stack(balanced_data['attention_mask'].values),
        np.array(balanced_data['labels'].tolist())
    )

# Балансування тренувальної вибірки
t_balanced_input_ids, t_balanced_attention_mask, t_balanced_labels = balance_data(
    t_toxic_input_ids, t_toxic_attention_mask, t_toxic_labels
)


In [37]:
# Перевірка розподілу класів
balanced_class_distribution = t_balanced_labels.sum(axis=0)
print("Розподіл класів у збалансованій вибірці:", balanced_class_distribution)


Розподіл класів у збалансованій вибірці: [26773.  9178. 21603.  5938. 21243.  8351.]


In [38]:
# Модель для мультиміткової класифікації, навчена на токсичних коментарях 

# Вхідні дані
input_ids = Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# BERT шар
bert_outputs = BertLayer(trainable=False)([input_ids, attention_mask])

# Пулінг
pooled_output = GlobalAveragePooling1D()(bert_outputs)

# багатоміткова класифікація токсичних коментарів
multilabel_dense = Dense(128, activation="swish")(pooled_output)
multilabel_dropout = Dropout(0.3)(multilabel_dense)
multilabel_output = Dense(6, activation="sigmoid", name="multilabel_output")(multilabel_dropout)

# Модель
model_3_2 = Model(
    inputs=[input_ids, attention_mask],
    outputs=[multilabel_output]
)

# Компіляція моделі без кастомної функції втрат
model_3_2.compile(
    optimizer=Adam(learning_rate=1e-4),  
    loss="binary_crossentropy",  
    metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), f1_metric]
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [39]:
# Навчання моделі
history_3_2 = model_3_2.fit(
    {
        'input_ids': t_balanced_input_ids,
        'attention_mask': t_balanced_attention_mask
    },
    t_balanced_labels,
    validation_data=(
        {
            'input_ids': v_toxic_input_ids,
            'attention_mask': v_toxic_attention_mask
        },
        v_toxic_labels),
    epochs=3,  
    batch_size=32
)

Epoch 1/3
876/876 ━━━━━━━━━━━━━━━━━━━━ 251s 266ms/step - accuracy: 0.8784 - f1_metric: 0.7782 - loss: 0.5231 - precision: 0.8155 - recall: 0.7452 - val_accuracy: 0.9420 - val_f1_metric: 0.7589 - val_loss: 0.5197 - val_precision: 0.6561 - val_recall: 0.9026
Epoch 2/3
876/876 ━━━━━━━━━━━━━━━━━━━━ 221s 252ms/step - accuracy: 0.9542 - f1_metric: 0.7859 - loss: 0.5085 - precision: 0.8282 - recall: 0.7483 - val_accuracy: 0.9420 - val_f1_metric: 0.7595 - val_loss: 0.5270 - val_precision: 0.6562 - val_recall: 0.9042
Epoch 3/3
876/876 ━━━━━━━━━━━━━━━━━━━━ 224s 256ms/step - accuracy: 0.9547 - f1_metric: 0.7876 - loss: 0.5050 - precision: 0.8275 - recall: 0.7519 - val_accuracy: 0.9420 - val_f1_metric: 0.7592 - val_loss: 0.4974 - val_precision: 0.6552 - val_recall: 0.9051


In [40]:
# Прогнози мультиміткової моделі на валідаціних даних
multilabel_predictions = model_3_2.predict(
    {'input_ids': v_toxic_input_ids, 'attention_mask': v_toxic_attention_mask},
    batch_size=64
)

# Перетворюємо на бінарні мітки
multilabel_predictions = (multilabel_predictions > 0.5).astype(int)

# Сумуємо значення для кожної мітки
toxic_label_counts = multilabel_predictions.sum(axis=0)

# Мітки токсичності
labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

print("\nРозподіл міток багатоміткової моделі:")
for i, label in enumerate(labels):
    print(f"{label}: {toxic_label_counts[i]}")

51/51 ━━━━━━━━━━━━━━━━━━━━ 34s 561ms/step

Розподіл міток багатоміткової моделі:
toxic: 3244
severe_toxic: 40
obscene: 3244
threat: 0
insult: 3244
identity_hate: 0


In [17]:
# Оцінка моделі
print("\nКласифікаційний звіт для другого підходу моделі:\n")
print(classification_report(v_toxic_labels, multilabel_predictions, target_names=[
    "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"
]))


Класифікаційний звіт для другого підходу моделі:

               precision    recall  f1-score   support

        toxic       0.94      1.00      0.97       931
 severe_toxic       0.00      0.00      0.00       105
      obscene       0.53      1.00      0.70       528
       threat       0.00      0.00      0.00        20
       insult       0.51      1.00      0.67       502
identity_hate       0.00      0.00      0.00        95

    micro avg       0.66      0.90      0.76      2181
    macro avg       0.33      0.50      0.39      2181
 weighted avg       0.65      0.90      0.74      2181
  samples avg       0.66      0.94      0.73      2181



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Результати моделі все ще низькі, проте певний прогрес є - модель визначила принаймні декілька прикладів рідкого класу severe_toxic. 

Можна очікувати, що при навчані моделі більшу кількість епох із раньої зупинкою та із фінтюнінгом, модель навчиться краще розпізнавати рідкі коментарі. 

Тому у якості архітектури використаємо останні пропозиції - відмова від кастомної функції втрат з вагами класів і балансування даних для навчання. 